# CPRN CREATE : PHASE A1 : RRL PATCH PROCESSING
**道路修补数据处理(osm based)**

`version` : `4.1.6`   
`author` : seika <seika@live.ca>   
`create_date` : `2025-09-09`   
`last_update` : `2025-10-28`   

---

Notes : 
+ CPRN构建的第一阶段，用于将 OSM 手动提取的高速公路补丁数据进行清洗、标准化和拓扑修复，生成可用于路网构建的标准化道路补丁数据
+ 输入的道路(手动选取OSM)存放在 `PATH_GPKG_RRL_PATCH` (gpkg) 文件中, 以不同图层代表需要补充的道路
+ 输出的标准化补丁道路, 位于 `PATH_SQLITE_CPRN` (spatialite) 文件中, 以 `LAYER_NAME_RRL_PATCH` 图层存储

---

Updates 
 + 2025-10-11 , patch 数据表名称统一
 + 2025-10-11 , 尝试添加 patch_link 表 (手动维护, 用于连接 patch 和 hd-rrl 的 link 关系(geom))
 + 2025-10-21 , 添加 G0040 部分 广陵-九华枢纽 区间改造后收费站路线 (使用 osm)

---


In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# --- SETTINGS ---
## PARAM: CRS
CRS_CGCS2, CRS_WGS84 = 'epsg:4549', 'epsg:4326'
## PARAM: DIST for fac snap/proj
MAX_SNAP_DIST, MAX_PROJ_DIST = 500, 50       # 设置最大吸附距离, 单位: 米
## PARAM: CONSTANT
CONST_HEIGHT = 00.00     # 标准海平面高度 (单位: 米)

## EXTENSIONS
PATH_SQLITE_EXT : str = r'/opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib'

## 🌀 INPUTS        # GPKG File that contains manually edited patch roads
PATH_GPKG_RRL_PATCH : str = r"/Users/seika/sync/lab_works/ZSTORE/geodata/geopackage/hdrrl_patch.gpkg"

## 💫 OUTPUTS
PATH_SQLITE_CPRN : str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite'
LAYER_NAME_RRL_PATCH : str = 'CPRN_rrl_patch_v416_jiangsu'


## 1 数据加载

In [2]:
# 🅰️③ 获取 hdrrl_patch 数据

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

lst_pat_lay = fiona.listlayers(PATH_GPKG_RRL_PATCH)

gdf_hdroad_patch = gpd.GeoDataFrame()

for lay in lst_pat_lay:
    gdf_ = gpd.read_file(PATH_GPKG_RRL_PATCH, layer = lay)
    gdf_hdroad_patch = pd.concat([gdf_hdroad_patch, gdf_], axis=0)

In [3]:
lst_pat_lay

['G0015_siming_tollstation',
 'S0018_mainroad',
 'G0015_taicang_ext',
 'G1521_mainroad_ext',
 'S0017_mainroad',
 'S0030_mainroad_construction',
 'S0065_patch_osm_251010',
 'aux_patch_link',
 'G0040_Jiuhua_intc',
 'G0040_rugaogang_tollstation',
 'G0040_jiangan_tollstation',
 'G0040_jishi_tollstation',
 'G0040_guangling_tollstation']

## 2 数据处理
  - 处理 construction 来源数据
  - 缺失值处理与规整

In [4]:
# ====== 处理 construction 来源数据 ======
##  确保只修改 concat 后的 gdf_hdroad_patch 中符合条件的记录
## 首先检查是否存在 construction 字段，避免出错
if 'construction' in gdf_hdroad_patch.columns:
    # 创建条件mask：highway为'construction'且construction字段不为空
    mask = (gdf_hdroad_patch['highway'] == 'construction') & (gdf_hdroad_patch['construction'].notna())
    
    # 只对符合条件的记录进行修改
    gdf_hdroad_patch.loc[mask, 'fclass'] = gdf_hdroad_patch.loc[mask, 'construction']
    
    print(f"已修改 {mask.sum()} 条记录(construction)的 highway 字段")
else:
    print("数据中没有 construction 字段，无需修改")

# 验证修改结果
print("\n修改后 highway 字段的值分布：")
print(gdf_hdroad_patch['highway'].value_counts())

# ------ 
# 对 code 为空（NaN）的记录，按照 fclass 映射
# 定义映射关系
fclass_to_code = {
    'motorway': 5111,
    'motorway_link': 5131
}

# 创建条件mask：code为空
code_nan_mask = gdf_hdroad_patch['code'].isna()

# 对每个映射关系进行处理
for fclass, code_value in fclass_to_code.items():
    # 创建条件：code为空且fclass匹配
    mask = code_nan_mask & (gdf_hdroad_patch['fclass'] == fclass)
    
    # 执行映射
    gdf_hdroad_patch.loc[mask, 'code'] = code_value
    
    print(f"已将 {mask.sum()} 条 fclass='{fclass}' 的记录的 code 设置为 {code_value}")

# 验证修改结果
print(f"\n修改后 code 字段的空值数量：{gdf_hdroad_patch['code'].isna().sum()}")
print("\ncode 字段的值分布：")
print(gdf_hdroad_patch['code'].value_counts())

# 查看 fclass 和对应的 code 分布
print("\nfclass 和 code 的对应关系：")
print(gdf_hdroad_patch[['fclass', 'code']].value_counts().sort_index())

# ------

# 处理 oneway 和 bridge 字段，将 'yes' 替换为 'T'
fields_to_process = ['oneway', 'bridge']

for field in fields_to_process:
    if field in gdf_hdroad_patch.columns:
        # 创建条件mask：字段值为'yes'
        mask = gdf_hdroad_patch[field] == 'yes'
        
        # 执行替换
        gdf_hdroad_patch.loc[mask, field] = 'T'
        
        print(f"已将 {mask.sum()} 条记录的 {field} 字段从 'yes' 替换为 'T'")
    else:
        print(f"数据中没有 {field} 字段")

# 验证修改结果
print("\n修改后的字段值分布：")
for field in fields_to_process:
    if field in gdf_hdroad_patch.columns:
        print(f"\n{field} 字段分布：")
        print(gdf_hdroad_patch[field].value_counts())

gdf_hdroad_patch = gdf_hdroad_patch[['osm_id','code','fclass','name','ref',
    'oneway','maxspeed', 'layer', 'bridge', 'tunnel','geometry']]

# 填充空值
# 使用 replace() 同时处理多种空值情况
for field in ['oneway', 'bridge', 'tunnel']:
    if field in gdf_hdroad_patch.columns:
        gdf_hdroad_patch[field] = gdf_hdroad_patch[field].replace({
            'None': 'F',  # 字符串 'None'
            None: 'F',    # Python None
            pd.NA: 'F',   # pandas NA
            np.nan: 'F'   # numpy NaN
        })
        
        print(f"{field} 字段：所有空值类型已替换为 'F'")

# reorder and reset index
gdf_hdroad_patch = gdf_hdroad_patch.sort_values(['osm_id']).reset_index(drop=True)

已修改 210 条记录(construction)的 highway 字段

修改后 highway 字段的值分布：
highway
construction    210
Name: count, dtype: int64
已将 88 条 fclass='motorway' 的记录的 code 设置为 5111
已将 122 条 fclass='motorway_link' 的记录的 code 设置为 5131

修改后 code 字段的空值数量：34

code 字段的值分布：
code
5131.0    494
5111.0    438
5141.0      2
Name: count, dtype: int64

fclass 和 code 的对应关系：
fclass         code  
motorway       5111.0    438
motorway_link  5131.0    494
service        5141.0      2
Name: count, dtype: int64
已将 208 条记录的 oneway 字段从 'yes' 替换为 'T'
已将 84 条记录的 bridge 字段从 'yes' 替换为 'T'

修改后的字段值分布：

oneway 字段分布：
oneway
F    716
T    208
B      2
Name: count, dtype: int64

bridge 字段分布：
bridge
F    434
T    368
Name: count, dtype: int64
oneway 字段：所有空值类型已替换为 'F'
bridge 字段：所有空值类型已替换为 'F'
tunnel 字段：所有空值类型已替换为 'F'


In [5]:
# ====== 炸开 MultiLineString ======
# 统计原始数据
print(f"\n处理前：{len(gdf_hdroad_patch)} 条记录")
print("几何体类型分布：")
print(gdf_hdroad_patch.geometry.geom_type.value_counts())

# 炸开多部分几何体
gdf_hdroad_patch = gdf_hdroad_patch.explode(index_parts=False).reset_index(drop=True)

# 统计处理后数据
print(f"\n处理后：{len(gdf_hdroad_patch)} 条记录")
print("几何体类型分布：")
print(gdf_hdroad_patch.geometry.geom_type.value_counts())


处理前：968 条记录
几何体类型分布：
LineString         547
MultiLineString    421
Name: count, dtype: int64

处理后：968 条记录
几何体类型分布：
LineString    968
Name: count, dtype: int64


## 3 RRL连通性修复(道路端点+控制点空间分析与断开)

In [6]:
gdf_rrlp = gdf_hdroad_patch.copy()

## geom force to 3D | 线串坐标强制转3D (TODO: 使用 DEM数据 计算线串高度 如有可能)
gdf_rrlp['geom'] = gdf_rrlp['geometry'].force_3d(z = CONST_HEIGHT)
gdf_rrlp = gdf_rrlp.set_geometry('geom')
gdf_rrlp = gdf_rrlp.drop(columns = ['geometry'])

## create geom of WGS and CGCS | 创建线串 wgs 和 cgcs 坐标系下的线串
from taisl_sop.model.gdf.geometry import create_multi_crs_geometry
gdf_rrlp = create_multi_crs_geometry(gdf = gdf_rrlp,
    geom_col = 'geom', crs = {'geom_rrl_wgs':'EPSG:4326', 'geom_rrl_cgcs':'EPSG:4549'},
    keep_original = False, verbose = True)
gdf_rrlp = gdf_rrlp.sort_values(['osm_id']).reset_index(drop = True)
gdf_rrlp['idx_rrl'] = gdf_rrlp.index

##  提取道路端点表
from taisl_sop.model.junction.junc_proc import extract_gdf_end_points
gdf_rrlp_ep = extract_gdf_end_points(gdf_rrlp, geom_col = 'geom_rrl_cgcs')
gdf_rrlp_ep = create_multi_crs_geometry(gdf = gdf_rrlp_ep, geom_col = 'geom', 
    crs = {'geom_ep_wgs':CRS_WGS84, 'geom_ep_cgcs':CRS_CGCS2}, keep_original = False, verbose = False)

## 使用rrl ep 找出需要断开的 rrl (及其 cut point)
from taisl_sop.model.junction.junc_proc import sjoin_ep_rrl
cols_rrl_ep = ['idx_ep', 'geom_ep_wgs', 'geom_ep_cgcs',]
cols_rrl = ['idx_rrl', 'geom_rrl_wgs', 'geom_rrl_cgcs'] 
cols_sj_eprrl = ['idx_ep','idx_rrl', 'de9im_pt_line', 'de9im_line_pt', 
    'geom_ep_wgs', 'geom_ep_cgcs', 'geom_rrl_wgs', 'geom_rrl_cgcs']

gdf_sj_eprrl = sjoin_ep_rrl(gdf_rrlp_ep, gdf_rrlp, cols_ep = cols_rrl_ep, cols_rrl = cols_rrl, cols_sj = cols_sj_eprrl)
gdf_sj_eprrl_cut = gdf_sj_eprrl.query("de9im_pt_line == '0FFFFF102' and de9im_line_pt == '0F1FF0FF2' ")

from taisl_sop.model.junction.junc_proc import cut_gdf_rrl_pt
col_base_rrl = ['idx_rrl', 'osm_id', 'code', 'fclass', 'name', 'ref', 
    'oneway', 'maxspeed', 'layer', 'bridge', 'tunnel']

gdf_rrlp_cf = cut_gdf_rrl_pt(gdf_rrlp, gdf_sj_eprrl_cut, 
    col_idx = 'idx_rrl', col_geom_rrl = 'geom_rrl_cgcs', col_geom_pt = 'geom_ep_cgcs', 
    cols_rrl = col_base_rrl, crs_cut = CRS_CGCS2, verbose = True)

gdf_rrlp_cf = gdf_rrlp_cf.to_crs(CRS_WGS84)
gdf_rrlp_cf = gdf_rrlp_cf.reset_index(drop = True)
gdf_rrlp_cf['idx_rrl'] = gdf_rrlp_cf.index

2025-10-27 17:22:56.523 | INFO     | taisl_sop.model.junction.junc_proc:split_gdf_rrl:99 - len gdf_rrl:968;121/847


In [7]:
print(len(gdf_hdroad_patch)), print(len(gdf_rrlp_cf))
gdf_rrlp_cf.sample(4)

968
1118


,idx_rrl,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geom
10,10,1132421512,5111.0,motorway,如常高速,S30,T,120,None,F,F,"LINESTRING Z (120.08576 32.10667 0, 120.0876 3..."
280,280,1014512452,5111.0,motorway,如常高速,S30,T,120,None,F,F,"LINESTRING Z (120.05943 32.08035 0, 120.06105 ..."
481,481,1176702969,5111.0,motorway,徐明高速,S65,F,120,1,T,F,"LINESTRING Z (117.44957 34.46881 0, 117.45118 ..."
1093,1093,NaN,NaN,NaN,NaN,NaN,F,NaN,NaN,F,F,"LINESTRING Z (117.63076 34.06994 35.6703, 117...."


## 4 EDA / CHECK

In [8]:
gdf_hdroad_patch.sample(4)

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
690,1353676379,5111.0,motorway,滨淮高速,S18,F,120.0,1.0,T,F,"LINESTRING (119.9488 34.13728, 119.94915 34.13..."
645,1333767348,5111.0,motorway,如常高速,S30,T,120,1,T,F,"LINESTRING (119.90014 31.96558, 119.90004 31.9..."
101,1165053819,5111.0,motorway,徐明高速,S65,F,120,0,F,F,"LINESTRING (117.64192 34.14097, 117.64193 34.1..."
98,1165053816,5111.0,motorway,徐明高速,S65,F,120,1,T,F,"LINESTRING (117.62848 34.23104, 117.62893 34.2..."


## 5 持久化与取回 🌖🌞

In [9]:
# 🌖 持久化
print(LAYER_NAME_RRL_PATCH)
#🪬 road reffrence line patch connection fixed
gdf_rrlp_cf.to_file(PATH_SQLITE_CPRN, driver='SQLite', spatialite=True, 
                        layer=LAYER_NAME_RRL_PATCH, overwrite=True,
                        index=False, mode='w', encoding='utf-8')

CPRN_rrl_patch_v416_jiangsu


In [10]:
# 🌞 取回
from taisl_sop.data.base.spatialite import SpatialitePuller
from taisl_sop.data.cprn.road.rrl_patch import RrlPatchFetcher


LAYER_NAME_RRL_PATCH : str = 'CPRN_rrl_patch_v416_jiangsu'
sp = SpatialitePuller(path_sqlite=PATH_SQLITE_CPRN, path_spatialite_ext=PATH_SQLITE_EXT, 
            set_precision=True, decimal_precision=16, driver='sqlite', verbose=True)
        
rrl_patch_fetcher = RrlPatchFetcher(sp, LAYER_NAME_RRL_PATCH, verbose=True)

gdf_rrlp_cf = rrl_patch_fetcher.fetch_rrl_patches(verbose=True) #🪬 road reffrence line patch connection fixed

2025-10-27 17:22:56.686 | INFO     | taisl_sop.data.base.spatialite:__init__:42 - SpatialitePuller initiated with path_sqlite: /Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite
2025-10-27 17:22:56.686 | INFO     | taisl_sop.data.base.spatialite:__init__:43 - Path_spatialite_ext: /opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib
2025-10-27 17:22:56.686 | INFO     | taisl_sop.data.base.spatialite:__init__:45 - Set_precision: ✅, decimal_precision: 16
2025-10-27 17:22:56.686 | INFO     | taisl_sop.data.cprn.road.rrl_patch:parse_sql:83 - parsed SQL: 
 
        SELECT ogc_fid, osm_id, code, fclass, name, ref,
               oneway, maxspeed, layer, bridge, tunnel,
               ASText(GEOMETRY) AS wkt,
               UPPER(ST_GeoHash(GEOMETRY, 12)) AS patch_geohash
          FROM CPRN_rrl_patch_v416_jiangsu;
        
2025-10-27 17:22:56.700 | INFO     | taisl_sop.data.base.spatialite:_setup_precision:83 - SpatiaLite version: 5.1.0
2025-10-27 17:22:56.700 

Memory usage of df puller : gdf 0.12 MB, df None MB


In [11]:
gdf_rrlp_cf

,ogc_fid,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,wkt,patch_geohash,geom
0,1,1014512438,5111.0,motorway,如常高速,S30,T,120,3,T,F,"LINESTRING Z(119.9826495 32.0213189 0, 119.985...",WTTQ4X79FV3Y,"LINESTRING Z (119.98265 32.02132 0, 119.98521 ..."
1,2,1014512438,5111.0,motorway,如常高速,S30,T,120,3,T,F,"LINESTRING Z(120.0125563 32.0433662 0, 120.013...",WTTQ74133D6P,"LINESTRING Z (120.01256 32.04337 0, 120.01377 ..."
2,3,1014512438,5111.0,motorway,如常高速,S30,T,120,3,T,F,"LINESTRING Z(120.020308 32.0513278 0, 120.0263...",WTTQ75X84R0W,"LINESTRING Z (120.02031 32.05133 0, 120.02632 ..."
3,4,1014512444,5111.0,motorway,如常高速,S30,T,120,None,F,F,"LINESTRING Z(120.0421074 32.06977139999999 0, ...",WTTQ7WK8PZ2R,"LINESTRING Z (120.04211 32.06977 0, 120.04416 ..."
4,5,1014512444,5111.0,motorway,如常高速,S30,T,120,None,F,F,"LINESTRING Z(120.0441604 32.07123900000001 0, ...",WTTQ7WXMFM05,"LINESTRING Z (120.04416 32.07124 0, 120.04569 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,1114,None,NaN,None,None,None,F,None,None,F,F,LINESTRING Z(120.2866332365732 32.123164491150...,WTTW9PY6RBSM,"LINESTRING Z (120.28663 32.12316 15.8201, 120...."
1114,1115,None,NaN,None,None,None,F,None,None,F,F,"LINESTRING Z(120.2477454 32.1286713 0, 120.248...",WTTWB2CFYUWM,"LINESTRING Z (120.24775 32.12867 0, 120.24838 ..."
1115,1116,None,NaN,None,None,None,F,None,None,F,F,LINESTRING Z(120.2499845706571 32.128490299837...,WTTWB2FFYG9D,"LINESTRING Z (120.24998 32.12849 13.57283, 120..."
1116,1117,None,NaN,None,None,None,F,None,None,F,F,"LINESTRING Z(120.2407769 32.1298856 0, 120.238...",WTTWB15DV7ZJ,"LINESTRING Z (120.24078 32.12989 0, 120.23806 ..."
